In [6]:
import json
import time
import pathlib
import numpy as np
import pandas as pd
from time import gmtime, strftime

import boto3

import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput, FeatureStoreOutput
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep

In [7]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = "arn:aws:iam::114169298763:role/service-role/AmazonSageMaker-ExecutionRole-20200811T163008"
account_id = boto3.client('sts').get_caller_identity()["Account"]

random_state = 42

Using AWS Region: us-west-2


In [8]:
%store -r
%store

Stored variables and their in-db values:
approved_model_name                   -> 'sagemaker-tutorial-xgboost-smote'
best_job_hp                           -> {'_tuning_objective_metric': 'validation:f1', 'alp
clarify_bias_job_1_name               -> 'Clarify-Bias-2021-04-23-17-55-18-464'
clarify_bias_job_2_name               -> 'Clarify-Bias-2021-04-23-19-01-35-414'
data_prefix                           -> 'sagemaker-tutorial/data'
default_bucket                        -> 'sagemaker-us-west-2-114169298763'
endpoint_config_name                  -> 'sagemaker-tutorial-xgboost-smote-endpoint-config'
endpoint_name                         -> 'sagemaker-tutorial-xgboost-smote-endpoint'
header                                -> ['LABEL', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIA
hyperparameters                       -> {'max_depth': '5', 'eta': '0.2', 'gamma': '4', 'mi
local_data_dir                        -> '../data'
local_processed_path                  -> '../data/df_processed.csv'
local_ra

In [9]:
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", 'imblearn'])

0

In [10]:
train_instance_param = ParameterString(
    name="TrainingInstance",
    default_value="ml.m4.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

deploy_model_instance_type = "ml.m4.xlarge"

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")

In [11]:
!wget https://raw.githubusercontent.com/shashankprasanna/sagemaker-workshop-2021/main/notebooks/04_e2e_pipeline/preprocessing.py

--2021-04-23 20:52:25--  https://raw.githubusercontent.com/shashankprasanna/sagemaker-workshop-2021/main/notebooks/04_e2e_pipeline/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2224 (2.2K) [text/plain]
Saving to: ‘preprocessing.py’

preprocessing.py    100%[===================>]   2.17K  --.-KB/s    in 0s      

2021-04-23 20:52:25 (44.8 MB/s) - ‘preprocessing.py’ saved [2224/2224]



In [12]:
s3_client.upload_file(Filename='./preprocessing.py', Bucket=default_bucket, Key=f'{prefix}/code/preprocessing.py')

create_dataset_script_uri = f's3://{default_bucket}/{prefix}/code/preprocessing.py'


create_dataset_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=sagemaker_role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name='credit-create-dataset',
    sagemaker_session=sagemaker_session)

create_dataset_step = ProcessingStep(
    name='CreateDataset',
    processor=create_dataset_processor,
    inputs=[ProcessingInput(
                        source=s3_raw_data,
                        destination='/opt/ml/processing/input')],
    outputs=[ProcessingOutput(output_name='train_data', source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name='test_data',  source='/opt/ml/processing/output/test')],
    job_arguments=["--train-test-split-ratio", '0.8'],
    code=create_dataset_script_uri)

In [13]:
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
content_type = "text/csv"
job_name = f'XgboostTrain-' + strftime('%d-%H-%M-%S', gmtime())
training_job_output_path = f's3://{default_bucket}/{prefix}/training_jobs'

In [14]:
use_spot_instances = False
max_run = 3600
max_wait = 7200 if use_spot_instances else None
checkpoint_s3_uri = (f's3://{default_bucket}/{prefix}/checkpoints/{job_name}' if use_spot_instances
                      else None)
print("Checkpoint path:", checkpoint_s3_uri)

Checkpoint path: None


In [15]:
!pip install sagemaker-experiments

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 42 kB 88 kB/s  eta 0:00:011


In [27]:
# construct a SageMaker estimator that calls the xgboost-container
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")

xgb_estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                              hyperparameters=best_job_hp,
                                              role="arn:aws:iam::114169298763:role/service-role/AmazonSageMaker-ExecutionRole-20200811T163008",
                                              instance_count=train_instance_count,
                                              instance_type=train_instance_type,
                                              volume_size=5,  # 5 GB
                                              output_path=training_job_output_path,
                                              use_spot_instances=use_spot_instances,
                                              max_run=max_run,
                                              max_wait=max_wait,
                                              checkpoint_s3_uri=checkpoint_s3_uri
                                             )


train_step = TrainingStep(
    name=job_name,
    estimator=xgb_estimator,
    inputs={
        'train': TrainingInput(
            s3_data=create_dataset_step.properties.ProcessingOutputConfig.Outputs['train_data'].S3Output.S3Uri,
        content_type="csv")
    }
)

In [17]:
model = sagemaker.model.Model(
    name='credit-default-demo-pipeline-xgboost',
    image_uri=train_step.properties.AlgorithmSpecification.TrainingImage,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=sagemaker_role
)

inputs = sagemaker.inputs.CreateModelInput(
    instance_type="ml.m4.xlarge"
)

create_model_step = CreateModelStep(
    name="ModelPreDeployment",
    model=model,
    inputs=inputs
)

In [18]:
# clarify config
bias_report_output_path = f's3://{default_bucket}/{prefix}/clarify-output/bias'
s3_client = boto3.client('s3', region_name=region)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=create_dataset_step.properties.ProcessingOutputConfig.Outputs['train_data'].S3Output.S3Uri,
    label='LABEL',
    dataset_type='text/csv',
    s3_output_path=bias_report_output_path)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name='SEX',
    facet_values_or_threshold=[1])

analysis_config = bias_data_config.get_config()
analysis_config.update(bias_config.get_config())
analysis_config["methods"] = {"pre_training_bias": {"methods": "all"}}

clarify_config_dir = pathlib.Path('config')
clarify_config_dir.mkdir(exist_ok=True)
with open(clarify_config_dir / 'analysis_config.json', 'w') as f:
    json.dump(analysis_config, f)

s3_client.upload_file(Filename='config/analysis_config.json', Bucket=default_bucket,
                      Key=f'{prefix}/clarify-config/analysis_config.json')

In [28]:
# clarify processing step
clarify_processor = sagemaker.processing.Processor(
    base_job_name='fraud-detection-demo-clarify-processor',
    image_uri=sagemaker.clarify.image_uris.retrieve(framework='clarify', region=region),
    role="arn:aws:iam::114169298763:role/service-role/AmazonSageMaker-ExecutionRole-20200811T163008",
    instance_count=1,
    instance_type='ml.c5.xlarge')

clarify_step = ProcessingStep(
    name="ClarifyProcessor",
    processor=clarify_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="analysis_config",
            source=f's3://{default_bucket}/{prefix}/clarify-config/analysis_config.json',
            destination="/opt/ml/processing/input/config"),
        sagemaker.processing.ProcessingInput(
            input_name="dataset",
            source=create_dataset_step.properties.ProcessingOutputConfig.Outputs['train_data'].S3Output.S3Uri,
            destination="/opt/ml/processing/input/data")
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            source="/opt/ml/processing/output/analysis.json",
            destination=bias_report_output_path,
            output_name="analysis_result")
    ]
)

In [29]:
class ModelMetrics(object):
    """Accepts model metrics parameters for conversion to request dict."""

    def __init__(
            self,
            model_statistics=None,
            model_constraints=None,
            model_data_statistics=None,
            model_data_constraints=None,
            bias=None,
            explainability=None,
    ):
        """Initialize a ``ModelMetrics`` instance and turn parameters into dict.
        Args:
            model_constraints (MetricsSource):
            model_data_constraints (MetricsSource):
            model_data_statistics (MetricsSource):
            bias (MetricsSource):
            explainability (MetricsSource):
        """
        self.model_statistics = model_statistics
        self.model_constraints = model_constraints
        self.model_data_statistics = model_data_statistics
        self.model_data_constraints = model_data_constraints
        self.bias = bias
        self.explainability = explainability

    def _to_request_dict(self):
        """Generates a request dictionary using the parameters provided to the class."""
        model_metrics_request = {}

        model_quality = {}
        if self.model_statistics is not None:
            model_quality["Statistics"] = self.model_statistics._to_request_dict()
        if self.model_constraints is not None:
            model_quality["Constraints"] = self.model_constraints._to_request_dict()
        if model_quality:
            model_metrics_request["ModelQuality"] = model_quality

        model_data_quality = {}
        if self.model_data_statistics is not None:
            model_data_quality["Statistics"] = self.model_data_statistics._to_request_dict()
        if self.model_data_constraints is not None:
            model_data_quality["Constraints"] = self.model_data_constraints._to_request_dict()
        if model_data_quality:
            model_metrics_request["ModelDataQuality"] = model_data_quality

        if self.bias is not None:
            model_metrics_request["Bias"] = {"Report": self.bias._to_request_dict()}
            # model_metrics_request["Bias"] = self.bias._to_request_dict()
        if self.explainability is not None:
            model_metrics_request["Explainability"] = self.explainability._to_request_dict()
        return model_metrics_request

In [30]:
model_metrics = ModelMetrics(
    bias=sagemaker.model_metrics.MetricsSource(
        s3_uri=clarify_step.properties.ProcessingOutputConfig.Outputs['analysis_result'].S3Output.S3Uri,
        content_type="application/json"
    )
)

if 'mpg_name' not in locals():
    mpg_name = prefix
    print(f'Model Package Group name: {mpg_name}')

register_step = RegisterModel(
    name="XgboostRegisterModel",
    estimator=xgb_estimator,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

In [31]:
!wget https://raw.githubusercontent.com/shashankprasanna/sagemaker-workshop-2021/main/notebooks/04_e2e_pipeline/deploy_model.py

--2021-04-23 21:22:56--  https://raw.githubusercontent.com/shashankprasanna/sagemaker-workshop-2021/main/notebooks/04_e2e_pipeline/deploy_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1871 (1.8K) [text/plain]
Saving to: ‘deploy_model.py.1’

deploy_model.py.1   100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-04-23 21:22:57 (24.6 MB/s) - ‘deploy_model.py.1’ saved [1871/1871]



In [32]:
endpoint_name = "xgboost-model-pipeline-" + strftime('%d-%H-%M-%S', gmtime())
deploy_model_script_uri = f's3://{default_bucket}/{prefix}/code/deploy_model.py'


s3_client.upload_file(Filename='deploy_model.py', Bucket=default_bucket, Key=f'{prefix}/code/deploy_model.py')

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=sagemaker_role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-detection-demo-deploy-model',
    sagemaker_session=sagemaker_session)

deploy_step = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model-name", create_model_step.properties.ModelName,
        "--region", region,
        "--endpoint-instance-type", deploy_model_instance_type,
        "--endpoint-name", endpoint_name],
    code=deploy_model_script_uri)

In [33]:
pipeline_name = f'credit-default'

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        train_instance_param,
        model_approval_status],
    steps=[
        create_dataset_step,
        train_step,
        create_model_step,
        clarify_step,
        register_step,
        deploy_step
    ])

In [34]:
pipeline.upsert(role_arn=sagemaker_role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-west-2:114169298763:pipeline/credit-default',
 'ResponseMetadata': {'RequestId': '80564696-4db4-4e82-a649-ad6d4010d7ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '80564696-4db4-4e82-a649-ad6d4010d7ee',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Fri, 23 Apr 2021 21:23:03 GMT'},
  'RetryAttempts': 0}}

In [35]:
start_response = pipeline.start()

start_response.wait()
start_response.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-west-2:114169298763:pipeline/credit-default',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-west-2:114169298763:pipeline/credit-default/execution/3zipd5m321aa',
 'PipelineExecutionDisplayName': 'execution-1619212986220',
 'PipelineExecutionStatus': 'Succeeded',
 'CreationTime': datetime.datetime(2021, 4, 23, 21, 23, 6, 137000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 4, 23, 21, 49, 48, 156000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-west-2:114169298763:user-profile/d-z78jqutsv0sx/adi-sagemaker-1600041260407',
  'UserProfileName': 'adi-sagemaker-1600041260407',
  'DomainId': 'd-z78jqutsv0sx'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-west-2:114169298763:user-profile/d-z78jqutsv0sx/adi-sagemaker-1600041260407',
  'UserProfileName': 'adi-sagemaker-1600041260407',
  'DomainId': 'd-z78jqutsv0sx'},
 'ResponseMetadata': {'RequestId': '1ade53e6-ccd1-4a51-9507-3d2159cfe90c',
  'HTT